# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))


1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text =  message.lower()
    
    # Replace URLs with a space in the message
    # from https://stackoverflow.com/questions/161738/what-is-the-best-regular-expression-to-check-if-a-string-is-a-valid-url
    url_regex = '((([A-Za-z]{3,9}:(?:\/\/)?)(?:[-;:&=\+\$,\w]+@)?[A-Za-z0-9.-]+|(?:www.|[-;:&=\+\$,\w]+@)[A-Za-z0-9.-]+)((?:\/[\+~%\/.\w\-_]*)?\??(?:[-\+=&;%@.\w_]*)#?(?:[\w]*))?)'
    
    text = re.sub(url_regex, ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    tickers = '\$\w+'
    text = re.sub(tickers, ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'@\w+', ' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-z]', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.strip().split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w) for w in tokens]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [6]:
# TODO Implement

tokenized = [preprocess(m) for m in messages]


In [7]:
from collections import Counter
"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
t_stack = [word for twit in tokenized for word in twit]
bow = Counter(t_stack)

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [8]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionary that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
total_num_words = len(t_stack)
freqs = {key: value/total_num_words for key, value in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = .000005

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 15

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [item[0] for item in bow.most_common(high_cutoff)]

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['the', 'to', 'a', 'is', 'for', 'on', 'of', 'and', 'in', 'this', 's', 'it', 'i', 'at', 'will']


6480

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [9]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: ii+1 for ii, word in enumerate(filtered_words)}
#print(vocab)
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 

id2vocab = {ii: word for word, ii in vocab.items()}
# tokenized with the words not in `filtered_words` removed.

#filtered = [[w for w in sublist if w in filtered_words] for sublist in tokenized]

filtered=[]
count = 0
for twit in tokenized:
    if count%(len(tokenized)/20) == 0:
        print(str((count)/len(tokenized)*100) + '%')
    filtered.append([w for w in twit if w in filtered_words])
    count+=1


0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [10]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [11]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19443080653362332

Finally let's convert our tokens into integer ids which we can pass to the network.

In [12]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [13]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, dropout=dropout)
        self.dropout = nn.Dropout(.3)
        
        self.fc = nn.Linear(lstm_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())

        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        
        batch_size = nn_input.size(1)
        x = nn_input.long()
        embed = self.embedding(x)
        lstm_output, hidden_state = self.lstm(embed, hidden_state)
        #lstm_output = lstm_output[-1]
        lstm_out = lstm_output[-1,:,:]  # https://knowledge.udacity.com/questions/30112  
    
        logps = self.softmax(self.dropout(self.fc(lstm_out)))
        #logps = logps[:, -1] # get last batch of labels
        
        
        
        return logps, hidden_state
        

### View Model

In [14]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.7516, -1.5273, -1.1707, -1.7828, -2.0318],
        [-1.2984, -1.5369, -1.7681, -1.7681, -1.7681],
        [-1.3372, -1.8172, -1.2279, -1.8433, -2.0895],
        [-1.3452, -1.8265, -1.2105, -1.8265, -2.1245]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [15]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [16]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   
import numpy as np
from sklearn.model_selection import train_test_split
# split size 8/2
train_features, valid_features, train_labels, valid_labels = train_test_split(token_ids, sentiments, test_size=.2, random_state=42)

In [17]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [18]:
import gc
# clear cuda memory at start
gc.collect()
torch.cuda.empty_cache()

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(6481, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (softmax): LogSoftmax()
)

In [20]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 2
# small batch size for memory allotment
batch_size = 128
sequence_length = 40
learning_rate = .003
clip=5 # gradient clipping

every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

best_val_acc = 0

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    hidden = model.init_hidden(batch_size)
    
    # split every train into batches using dataloader
    
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=sequence_length, shuffle=True):
        
        if text_batch.size()!=torch.Size([sequence_length,batch_size]):
            continue
        
        steps += 1
        
        # update hidden
        hidden = tuple([each.data for each in hidden])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # Train Model
        # per usual implementation
        model.zero_grad()
        
        log_ps, hidden = model.forward(text_batch, hidden)
        
        loss = criterion(log_ps, labels)
        
        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        if steps % every == 0:
            # test validation every 'every' steps
            model.eval()
            val_losses = []
            val_accuracy = []
            
            val_hidden = model.init_hidden(batch_size)
            
            for val_text_batch, val_labels in dataloader(
            valid_features, valid_labels, batch_size = batch_size, sequence_length = sequence_length):
                
                if val_text_batch.size() != torch.Size([sequence_length, batch_size]):
                    continue
                
                val_text_batch, val_labels = val_text_batch.to(device), val_labels.to(device)
                
                val_hidden = tuple([each.data for each in val_hidden])
                
                # move each validation hidden element to device
                for each in val_hidden:
                    each.to(device)
                val_log_ps, hidden = model.forward(val_text_batch, val_hidden)
                
                val_loss = criterion(val_log_ps.squeeze(), val_labels)
                val_losses.append(val_loss.item())
                
                val_ps = torch.exp(val_log_ps)
                top_p, top_class = val_ps.topk(1, dim=1)
                
                equals = top_class == val_labels.view(*top_class.shape)
                
                val_accuracy.append(torch.mean(equals.type(torch.FloatTensor)).item())
                #print(val_accuracy)
                
            
            val_acc = np.average(val_accuracy)
                
            model.train()
            
            
            # Print metrics
            
            # formatted printing!
            print("Epoch:{}/{}...".format(epoch+1, epochs),
                  "Step:{}...".format(steps),
                  "Loss:{:.6f}...".format(loss.item()),
                  "Val Loss:{:.6f}...".format(np.average(val_losses)),
                  "Val Accuracy:{:.6f}...".format(val_acc))
            
            # save best
            
            if val_acc > best_val_acc:
                torch.save({
                'epoch': epoch,
                'step':steps,
                #'model_state_dict': model.state_dic(),
                #'optimizer_state_dict': optimizer.stat_dic(),
                'loss': loss,
                }, 'best_model')
                best_val_acc = val_acc
                print("Saved model with new best accuracy")

            


Starting epoch 1
Epoch:1/2... Step:100... Loss:1.264938... Val Loss:1.130594... Val Accuracy:0.545280...
Saved model with new best accuracy
Epoch:1/2... Step:200... Loss:1.013031... Val Loss:0.993536... Val Accuracy:0.599866...
Saved model with new best accuracy
Epoch:1/2... Step:300... Loss:1.188803... Val Loss:0.950111... Val Accuracy:0.629983...
Saved model with new best accuracy
Epoch:1/2... Step:400... Loss:1.098745... Val Loss:0.902931... Val Accuracy:0.638291...
Saved model with new best accuracy
Epoch:1/2... Step:500... Loss:1.133217... Val Loss:0.879694... Val Accuracy:0.657640...
Saved model with new best accuracy
Epoch:1/2... Step:600... Loss:1.035459... Val Loss:0.877246... Val Accuracy:0.667023...
Saved model with new best accuracy
Epoch:1/2... Step:700... Loss:1.023107... Val Loss:0.848384... Val Accuracy:0.670485...
Saved model with new best accuracy
Epoch:1/2... Step:800... Loss:0.858570... Val Loss:0.836669... Val Accuracy:0.675166...
Saved model with new best accuracy

Epoch:2/2... Step:1800... Loss:0.996362... Val Loss:0.737098... Val Accuracy:0.714647...
Epoch:2/2... Step:1900... Loss:0.954977... Val Loss:0.743198... Val Accuracy:0.712999...
Epoch:2/2... Step:2000... Loss:0.912673... Val Loss:0.739591... Val Accuracy:0.715055...
Epoch:2/2... Step:2100... Loss:0.933201... Val Loss:0.742836... Val Accuracy:0.714059...
Epoch:2/2... Step:2200... Loss:0.970054... Val Loss:0.738109... Val Accuracy:0.714482...
Epoch:2/2... Step:2300... Loss:1.041809... Val Loss:0.735566... Val Accuracy:0.716528...
Saved model with new best accuracy
Epoch:2/2... Step:2400... Loss:0.861592... Val Loss:0.739176... Val Accuracy:0.715454...
Epoch:2/2... Step:2500... Loss:0.935277... Val Loss:0.737544... Val Accuracy:0.716810...
Saved model with new best accuracy
Epoch:2/2... Step:2600... Loss:0.952336... Val Loss:0.746323... Val Accuracy:0.717044...
Saved model with new best accuracy
Epoch:2/2... Step:2700... Loss:0.801818... Val Loss:0.734391... Val Accuracy:0.717209...
Saved

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [25]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in filtered_words]
    
    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.tensor(tokens).unsqueeze(1)

    # Get the NN output
    hidden = model.init_hidden(text_input.size(1))
    logps, _ = model.forward(text_input,hidden)
    
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred.detach().numpy()
    

In [33]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)
sentiment=predict(text, model, vocab)
print(sentiment)

[[ 0.00299778  0.04704447  0.03089876  0.62741584  0.29164314]]


### Questions: What is the prediction of the model? What is the uncertainty of the prediction?


The prediction of the model is that the sentiment label is +1, indicating a positive sentiment.
The model evaluates with a test accuracy of 62.74%, which indicates that there is still 47.26% of uncertainty in our prediction.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [34]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [35]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [36]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [37]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': array([[ 0.09126665,  0.10763841,  0.28618225,  0.20392765,  0.310985  ]], dtype=float32),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.